In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [2]:
def get_predictions(model, directory, target_dim):
    
    classes_list = os.listdir(directory)
    X = []
    y = []
    for target in classes_list:
        print("Predicting for class: " + str(target))
        images_list = os.listdir(directory + target)
        for img in images_list:
            
            image = tf.keras.preprocessing.image.load_img(directory + target + '/' + img, target_size=(target_dim, target_dim))
            image = tf.keras.preprocessing.image.img_to_array(image)
            
            image = image.reshape(1, target_dim, target_dim, 3)
            image = tf.keras.applications.vgg19.preprocess_input(image)
            image = image / 255
            
            pred = model.predict(image)[0]
            X.append(pred)
            y.append(target)
        
        print("Done predicting for class: " + str(target))
        
    return np.array(X), np.array(y).reshape(-1, 1)

In [3]:
def generate_dataset(model, target_dim):

    print("Training data")
    X_train, y_train = get_predictions(model, './training/', target_dim)
    print("Val data")
    X_val, y_val = get_predictions(model, './validation/', target_dim)
    
    return X_train, y_train, X_val, y_val

In [4]:
inception = tf.keras.models.load_model('./models/inceptionV3-regularized-04-0.83.hdf5')
vgg = tf.keras.models.load_model('./models/vgg19-regularized-08-0.78.hdf5')
mobilenet = tf.keras.models.load_model('./models/mobilenet-regularized-05-0.80.hdf5')

models = [inception, vgg, mobilenet]
target_dims = [300, 300, 224]
X_train = pd.DataFrame()
Y_train = pd.DataFrame()
X_val = pd.DataFrame()
Y_val = pd.DataFrame()

In [6]:
for i in range(3):
    
    print("Predicting using model " + str(i))
    x_train, y_train, x_val, y_val = generate_dataset(models[i], target_dims[i])
    X_train = pd.concat([X_train, pd.DataFrame(x_train)])
    X_val = pd.concat([X_val, pd.DataFrame(x_val)])
    Y_train = pd.concat([Y_train, pd.DataFrame(y_train)])
    Y_val = pd.concat([Y_val, pd.DataFrame(y_val)])
    

Predicting using model 0
Training data
Predicting for class: Bread
Done predicting for class: Bread
Predicting for class: Dairyproduct
Done predicting for class: Dairyproduct
Predicting for class: Dessert
Done predicting for class: Dessert
Predicting for class: Egg
Done predicting for class: Egg
Predicting for class: Friedfood
Done predicting for class: Friedfood
Predicting for class: Meat
Done predicting for class: Meat
Predicting for class: NoodlesPasta
Done predicting for class: NoodlesPasta
Predicting for class: Rice
Done predicting for class: Rice
Predicting for class: Seafood
Done predicting for class: Seafood
Predicting for class: Soup
Done predicting for class: Soup
Predicting for class: VegetableFruit
Done predicting for class: VegetableFruit
Val data
Predicting for class: Bread
Done predicting for class: Bread
Predicting for class: Dairyproduct
Done predicting for class: Dairyproduct
Predicting for class: Dessert
Done predicting for class: Dessert
Predicting for class: Egg
Do

In [7]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(29598, 11)
(10290, 11)
(29598, 1)
(10290, 1)


In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train).reshape(-1, 1)
Y_val = encoder.fit_transform(Y_val).reshape(-1, 1)

C:\Users\sutar\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(11, activation=tf.nn.softmax)
])

In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['acc']
)

In [13]:
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=5, batch_size=32)

Epoch 1/5
925/925 [==============================] - 1s 1ms/step - loss: 0.7852 - acc: 0.8414 - val_loss: 0.6533 - val_acc: 0.8247
Epoch 2/5
925/925 [==============================] - 1s 1ms/step - loss: 0.3164 - acc: 0.9122 - val_loss: 0.6571 - val_acc: 0.8261
Epoch 3/5
925/925 [==============================] - 1s 1ms/step - loss: 0.3112 - acc: 0.9126 - val_loss: 0.6515 - val_acc: 0.8251
Epoch 4/5
925/925 [==============================] - 1s 1ms/step - loss: 0.3096 - acc: 0.9128 - val_loss: 0.6567 - val_acc: 0.8270
Epoch 5/5
925/925 [==============================] - 1s 1ms/step - loss: 0.3091 - acc: 0.9129 - val_loss: 0.6532 - val_acc: 0.8255


In [14]:
X_test = pd.DataFrame()
Y_test = pd.DataFrame()

In [15]:
for i in range(3):
    
    print("Predicting with model: " + str(i))    
    x_test, y_test = get_predictions(models[i], './evaluation/', target_dims[i])
    X_test = pd.concat([X_test, pd.DataFrame(x_test)])
    Y_test = pd.concat([Y_test, pd.DataFrame(y_test)])
    

Predicting with model: 0
Predicting for class: Bread
Done predicting for class: Bread
Predicting for class: Dairyproduct
Done predicting for class: Dairyproduct
Predicting for class: Dessert
Done predicting for class: Dessert
Predicting for class: Egg
Done predicting for class: Egg
Predicting for class: Friedfood
Done predicting for class: Friedfood
Predicting for class: Meat
Done predicting for class: Meat
Predicting for class: NoodlesPasta
Done predicting for class: NoodlesPasta
Predicting for class: Rice
Done predicting for class: Rice
Predicting for class: Seafood
Done predicting for class: Seafood
Predicting for class: Soup
Done predicting for class: Soup
Predicting for class: VegetableFruit
Done predicting for class: VegetableFruit
Predicting with model: 1
Predicting for class: Bread
Done predicting for class: Bread
Predicting for class: Dairyproduct
Done predicting for class: Dairyproduct
Predicting for class: Dessert
Done predicting for class: Dessert
Predicting for class: Egg


In [17]:
Y_test = encoder.transform(Y_test).reshape(-1, 1)

C:\Users\sutar\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
model.evaluate(X_test, Y_test)

314/314 [==============================] - 0s 769us/step - loss: 0.5750 - acc: 0.8461


[0.5750025510787964, 0.8461308479309082]

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [38]:
rfc = RandomForestClassifier(n_estimators=100)

In [39]:
def fit_predict(model):
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)
    print(accuracy_score(Y_test, predictions.reshape(-1, 1)))

In [40]:
fit_predict(rfc)

C:\Users\sutar\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8549945224579225
